In [1]:
import sys
import importlib

def check_version(package_name):
    try:
        lib = importlib.import_module(package_name)
        print(f"✅ {package_name:<15} : {lib.__version__}")
    except ImportError:
        print(f"❌ {package_name:<15} : 未安装")
    except Exception as e:
        print(f"⚠️ {package_name:<15} : 导入出错 ({e})")

print(f"Python 版本: {sys.version.split()[0]}")
print("-" * 30)

# 检查核心依赖
pkgs = ['vllm', 'numpy', 'scipy', 'transformers', 'torch']
for pkg in pkgs:
    check_version(pkg)

Python 版本: 3.12.3
------------------------------
❌ vllm            : 未安装
✅ numpy           : 2.4.1
❌ scipy           : 未安装
✅ transformers    : 5.0.0.dev0
✅ torch           : 2.5.1+rocm6.2


In [ ]:
import os
import subprocess
import sys

# ================= ⚙️ 配置区域 =================
ENV_NAME = "env_glm"
DISPLAY_NAME = "🎨 GLM-Image (Artist)"

# 需要安装的绘图依赖 (完全不含 torch)
# 注意：我们将强制安装 numpy 1.26.4 到这个虚拟环境里，
# 这样它会“遮挡”住系统原本的 numpy 2.1，但不会影响系统原本的 torch。
requirements = [
    "numpy==1.26.4", 
    "diffusers", 
    "transformers", 
    "accelerate", 
    "sentencepiece", 
    "opencv-python-headless", 
    "pillow", 
    "ipykernel"
]

# ================= 🚀 自动化脚本 =================

def run_cmd(cmd, desc):
    print(f"⏳ {desc}...")
    try:
        subprocess.run(cmd, shell=True, check=True, executable="/bin/bash")
        print("   ✅ 成功")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ 失败: {e}")
        raise

print(f"🚀 开始构建 GLM-Image 环境 (继承现有 PyTorch)...\n")

# 1. 创建虚拟环境 (关键参数: --system-site-packages)
# 这会让新环境直接“看到”您基础环境里的 PyTorch，而不需要重新安装
if not os.path.exists(ENV_NAME):
    cmd_venv = f"{sys.executable} -m venv {ENV_NAME} --system-site-packages"
    run_cmd(cmd_venv, "创建继承式虚拟环境")
else:
    print(f"   ⚠️ 环境目录 {ENV_NAME} 已存在，跳过创建")

# 获取路径
env_bin = os.path.abspath(os.path.join(ENV_NAME, "bin"))
env_pip = os.path.join(env_bin, "pip")
env_python = os.path.join(env_bin, "python")

# 2. 局部安装 Numpy 1.26 (关键步骤)
# 使用 --ignore-installed 强制在虚拟环境里装一个旧版 numpy
# 这就像给新环境戴了一副眼镜，让它看到的是 1.26，而不是系统的 2.1
print(f"⏳ 正在配置兼容性 Numpy...")
cmd_numpy = f"{env_pip} install numpy==1.26.4 --ignore-installed"
run_cmd(cmd_numpy, "Numpy 1.26.4 覆盖安装")

# 3. 安装 Diffusers 等库
# pip 会检测到系统里已经有 torch 了，所以它只需安装剩下的库
pkgs_str = " ".join(requirements[1:]) # 跳过列表第一个numpy(已装)
print(f"⏳ 安装绘图依赖 (Diffusers, Transformers)...")
cmd_deps = f"{env_pip} install {pkgs_str}"
run_cmd(cmd_deps, "依赖库安装")

# 4. 注册内核
cmd_kernel = f'{env_python} -m ipykernel install --user --name={ENV_NAME} --display-name "{DISPLAY_NAME}"'
run_cmd(cmd_kernel, "内核注册")

# 5. 最终验证
print("\n" + "="*50)
print("🔍 验证环境隔离性与继承性...")
verify_script = """
import torch
import numpy
import sys

print(f'Python 路径: {sys.executable}')
print(f'PyTorch 版本: {torch.__version__} (继承自系统)')
print(f'ROCm 状态:    {torch.version.hip}')
print(f'Numpy 版本:   {numpy.__version__} (虚拟环境独有)')

# 检查逻辑
if 'rocm' not in torch.__version__ and not torch.version.hip and 'git' not in torch.__version__:
     # 注：您的版本含 git 字符，所以加个判断
    print('⚠️ 警告：PyTorch 来源看起来不对，请检查。')
elif numpy.__version__ != '1.26.4':
    print('❌ 错误：Numpy 版本没锁住！')
else:
    print('✅ 完美！成功复用了系统的 ROCm PyTorch，同时使用了兼容的 Numpy 1.26。')
"""

try:
    subprocess.run([env_python, "-c", verify_script], check=True)
    print("🎉🎉🎉 环境构建完成！请刷新页面切换内核。")
except subprocess.CalledProcessError:
    print("❌ 验证脚本报错，请检查日志。")

In [ ]:
import os
from huggingface_hub import snapshot_download

# ================= 🔧 配置 =================
# 1. 强制使用国内镜像 (HF-Mirror)
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 2. 定义下载路径
repo_id = "zai-org/GLM-Image"
local_model_path = "/workspace/models/GLM-Image"

print(f"🚀 开始从 hf-mirror.com 下载模型: {repo_id}")
print(f"📂 目标保存路径: {local_model_path}")
print("⚠️ 模型体积约 30GB+，请耐心等待...")

try:
    # ================= 📥 开始下载 =================
    # snapshot_download 是 Python 版的 "huggingface-cli download"
    snapshot_download(
        repo_id=repo_id,
        local_dir=local_model_path,
        local_dir_use_symlinks=False, # 直接下载文件，方便查看
        resume_download=True,         # 支持断点续传
        max_workers=8                 # 多线程加速
    )
    
    print("\n" + "="*40)
    print("✅ 模型下载成功！")
    print(f"你现在可以运行 [第 3 步] 加载模型了。")
    print("="*40)

except ImportError:
    print("\n❌ 报错：缺少 huggingface_hub 库。")
    print("请先运行: !pip install huggingface_hub -i https://pypi.tuna.tsinghua.edu.cn/simple")
except Exception as e:
    print(f"\n❌ 下载失败: {e}")
    print("💡 提示: 网络可能中断，请直接重新运行此单元格继续下载。")

In [ ]:
import os

print("🗑️ 正在卸载 bitsandbytes (它是导致报错的元凶)...")
os.system("pip uninstall bitsandbytes -y")
print("✅ 卸载完成！")

# ⚠️ 关键：为了确保 Python 重新加载环境，建议重启内核
# 但如果不想重启，我们可以尝试继续运行下一步，通常也没问题

In [ ]:
import os
import time

print("🚑 正在修复库版本冲突...")

# 1. 卸载可能冲突的旧版 peft
os.system("pip uninstall peft -y")

# 2. 从 GitHub 源码安装最新版 peft (使用加速镜像)
# 这样能确保它和 transformers 5.0.0.dev0 完美兼容
exit_code = os.system("pip install git+https://gh-proxy.org/https://github.com//huggingface/peft.git -i https://pypi.tuna.tsinghua.edu.cn/simple")

if exit_code == 0:
    print("\n✅ PEFT 修复成功！")
    print("⚠️ 关键提示：为了应用更改，请务必【重启内核】(Restart Kernel)！")
    print("   (在 Notebook 菜单栏点击：Kernel -> Restart Kernel)")
else:
    print("\n❌ PEFT 安装失败，请检查网络。")

In [ ]:
import os

print("🚑 正在修复 Tokenizer 依赖...")

# 1. 强制升级 mistral_common 到最新版
# 2. 同时更新 tiktoken (GLM-Image 强依赖这个)
os.system("pip install -U mistral_common tiktoken -i https://pypi.tuna.tsinghua.edu.cn/simple")

print("✅ 修复完成！")
print("⚠️ 请务必【重启内核 (Restart Kernel)】后再运行下一步！")

In [ ]:
import os

print("🚑 正在执行深度修复 (Deep Repair)...")
print("🔄 正在切换镜像源至: https://gh-proxy.org/")

# 1. 🗑️ 卸载冲突包 (清理旧环境)
# 必须把 bitsandbytes 卸载掉，否则会报 ROCm 错误
pkgs_to_remove = "bitsandbytes peft transformers diffusers accelerate"
print(f"   ↳ 正在卸载旧版本: {pkgs_to_remove} ...")
os.system(f"pip uninstall -y {pkgs_to_remove}")

# 2. 📦 重新安装 (使用新镜像源)
print("   ↳ 正在从源码重新安装依赖 (请耐心等待)...")

install_cmds = [
    # 基础依赖 (走清华源)
    "pip install -U huggingface_hub sentencepiece protobuf requests -i https://pypi.tuna.tsinghua.edu.cn/simple",
    
    # 核心库 (走 gh-proxy.org 加速)
    # 注意：peft 必须装开发版才能配合最新的 transformers
    "pip install git+https://gh-proxy.org/https://github.com/huggingface/peft.git",
    "pip install git+https://gh-proxy.org/https://github.com/huggingface/accelerate.git",
    "pip install git+https://gh-proxy.org/https://github.com/huggingface/transformers.git",
    "pip install git+https://gh-proxy.org/https://github.com/huggingface/diffusers.git"
]

for cmd in install_cmds:
    print(f"      执行: {cmd}")
    os.system(cmd)

print("\n✅ 环境修复完成！")
print("⚠️⚠️⚠️ 请务必点击上方菜单栏【Kernel -> Restart Kernel】重启内核！⚠️⚠️⚠️")

In [ ]:
import os

print("🚑 正在移除冲突的 Mistral 依赖库...")
# 强制卸载 mistral_common，它阻碍了 GLM-Image 的加载
os.system("pip uninstall -y mistral_common")

print("✅ 冲突已清除！")
print("⚠️⚠️⚠️以此为界：请务必点击上方菜单【Kernel -> Restart Kernel】重启内核！⚠️⚠️⚠️")

In [ ]:
import torch
import gc
import os
import warnings
import logging
from datetime import datetime
from tqdm.auto import tqdm
from diffusers import GlmImagePipeline

# ================= 1. 告警屏蔽 (保持清爽) =================
# 使用 r"" 原始字符串避免 SyntaxWarning
warnings.filterwarnings("ignore", message=".*Passing `generation_config` together with.*")
warnings.filterwarnings("ignore", message=r".*calling .generate\(\) with the `input_ids` being on a device.*")
warnings.filterwarnings("ignore", message=".*Attempting to use hipBLASLt on an unsupported architecture!.*")
logging.getLogger("transformers").setLevel(logging.ERROR)

print("🛡️ 系统初始化中...")

# ================= 2. 自动目录管理 (新增功能) =================
# 定义基础保存路径
base_output_dir = "glm_gallery"

# 获取当前时间
now = datetime.now()
date_folder = now.strftime("%Y-%m-%d")  # 例如: 2026-01-16
time_filename = now.strftime("%H-%M-%S") # 例如: 09-30-01

# 创建按日期分类的文件夹
save_dir = os.path.join(base_output_dir, date_folder)
os.makedirs(save_dir, exist_ok=True)

print(f"📂 图片将保存至: {save_dir}")

# ================= 3. 显存清理 =================
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

# ================= 4. 加载模型 =================
model_path = "/workspace/models/GLM-Image"
print(f"🚀 正在加载 GLM-Image...")

try:
    pipe = GlmImagePipeline.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        use_safetensors=True
    )

    # MI50 必选补丁 (防黑图)
    if hasattr(pipe, "vqmodel"):
        pipe.vqmodel.to(dtype=torch.float32)
    elif hasattr(pipe, "vae"):
        pipe.vae.to(dtype=torch.float32)

    # 32G 显存必选优化 (CPU Offload)
    pipe.enable_model_cpu_offload()
    print("✅ 模型加载完毕！")

    # ================= 5. 进度条回调 =================
    class ProgressBar:
        def __init__(self, total_steps):
            self.pbar = tqdm(total=total_steps, desc="🎨 绘制进度", unit="step")
        
        def __call__(self, pipe, step, timestep, callback_kwargs):
            self.pbar.update(1)
            return callback_kwargs

    # ================= 6. 生成图片 =================
    # 提示词
    prompt = "A steampunk hamster mechanic fixing a clockwork robot, brass gears, steam, highly detailed, 8k"
    steps = 30
    
    print(f"🖌️ 任务开始: {prompt}")

    progress_callback = ProgressBar(total_steps=steps)

    with torch.no_grad():
        image = pipe(
            prompt=prompt,
            height=768,
            width=768,
            num_inference_steps=steps,
            guidance_scale=5.0,
            callback_on_step_end=progress_callback,
        ).images[0]
    
    progress_callback.pbar.close()

    # ================= 7. 保存结果 =================
    # 构造完整文件名
    full_save_path = os.path.join(save_dir, f"img_{time_filename}.png")
    
    image.save(full_save_path)
    print(f"\n🎉 成功！已归档至: {full_save_path}")
    
    from IPython.display import display
    display(image)

except Exception as e:
    print(f"\n❌ 发生错误: {e}")